In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [33]:
cred = pd.read_csv("credit_activities.csv")
proj = pd.read_csv("project_info.csv")
prev = pd.read_csv("../RawData.csv")

/var/folders/gn/x53b3rps7jz6jbl5j86y21gr0000gn/T/ipykernel_36324/1122147834.py:3: DtypeWarning: Columns (12,18,19,22,23,24,25,48,59,60,61,77,79,80,81,82,105,106,108,111,114,115,116,117,122,129,132,133,134,135,151) have mixed types. Specify dtype option on import or set low_memory=False.
  prev = pd.read_csv("../RawData.csv")


In [37]:
prev = prev[['Project ID', 'Scope', ' Type', 'Reduction / Removal']]
prev.columns = ['Project ID', 'Scope', 'Type', 'Reduction/Removal']

KeyError: "[' Type', 'Reduction / Removal'] not in index"

In [38]:
proj.columns

Index(['project_id', 'project_name', 'voluntary_registry', 'arb_wa_project',
       'voluntary_status', 'scope', 'type', 'type_rule', 'reduction_removal',
       'methodology_protocol', 'methodology_version', 'region', 'country',
       'state', 'project_site_location', 'project_developer', 'project_owner',
       'offset_project_operator', 'authorized_project_designee', 'verifier',
       'estimated_annual_emission_reductions', 'arb_id', 'arb_project_type',
       'arb_project_status', 'wa_id', 'wa_project_type', 'poa_id',
       'project_listed', 'project_registered', 'sustainability_certification',
       'project_type_from_the_registry', 'registry_documents',
       'project_website', 'notes_from_registry', 'notes_from_bctp',
       'project_description', 'added_to_database_version'],
      dtype='object')

In [42]:
proj_names = ['Project ID', 'Project Name', 'Voluntary Registry', 'ARB/WA Project', 
              'Voluntary Status', 'Scope_new', 'Type_new', 'Type Rule', 'Reduction/Removal_new', 
              'Methodology/Protocol', 'Methodology Version', 'Region', 'Country',
              'State', 'Project Site Location', 'Project Developer', 'Project Owner',
              'Offset Project Operator', 'Authorized Project Designee', 'Verifier',
              'Estimated Annual Emission Reductions', 'ARB ID', 'ARB Project Type',
              'ARB Project Status', 'WA ID', 'WA Project Type', 'POA ID', 
              'Project Listed', 'Project Registered', 'Sustainability Certification',
              'Project Type From the Registry', 'Registry Documents',
              'Project Website', 'Notes From Registry', 'Notes from BCTP', 
              'Project Description', 'Added to DB Version']

In [43]:
proj.columns = proj_names
proj1 = proj.iloc[:, :17]
proj2 = proj.drop(proj.columns[1:17], axis=1)

In [6]:
proj2.columns

Index(['Project ID', 'Offset Project Operator', 'Authorized Project Designee',
       'Verifier', 'Estimated Annual Emission Reductions', 'ARB ID',
       'ARB Project Type', 'ARB Project Status', 'WA ID', 'WA Project Type',
       'POA ID', 'Project Listed', 'Project Registered',
       'Sustainability Certification', 'Project Type From the Registry',
       'Registry Documents', 'Project Website', 'Notes From Registry',
       'Notes from BCTP', 'Project Description', 'Added to DB Version'],
      dtype='object')

In [7]:
cred.transaction_type.unique()

array(['issuance', 'buffer_contribution', 'retirement_cancellation',
       'buffer_subtraction', 'per'], dtype=object)

In [8]:
cred[['vintage_year','issuance_date']] = cred[['vintage_year','issuance_date']].fillna(0)
#cred['vintage_year'] = cred['vintage_start'].str[:4]
cred['issuance_date'] = cred['date'].str[:4]
cred.loc[cred['issuance_date'].str.startswith('12/1'), 'issuance_date'] = '2023'
cred.loc[cred['issuance_date'].str.startswith('2/1/'), 'issuance_date'] = '2024'

In [9]:
cred_group = cred.groupby(['project_id', 'transaction_type', 'issuance_date']).amount.sum().reset_index()

In [10]:
cred_piv = cred_group.pivot(index=['project_id', 'issuance_date'], 
                      columns='transaction_type', values='amount').reset_index()

In [11]:
cred_piv.head()

transaction_type,project_id,issuance_date,buffer_contribution,buffer_subtraction,issuance,per,retirement_cancellation
0,ACR102,2005,NaN,NaN,1898000.0,NaN,NaN
1,ACR102,2006,NaN,NaN,6086006.0,NaN,NaN
2,ACR102,2011,NaN,NaN,NaN,NaN,5000.0
3,ACR102,2012,NaN,NaN,NaN,NaN,1966.0
4,ACR102,2013,NaN,NaN,NaN,NaN,400.0


In [12]:
cred_group = cred_piv.groupby(['project_id']).agg({'issuance_date':'min',
                                               'buffer_contribution':'sum', 
                                               'buffer_subtraction':'sum', 
                                               'issuance':'sum',
                                               'retirement_cancellation':'sum'}).reset_index()

In [13]:
cred_group.columns = ['Project ID', 'Issuance Date', 'Total Buffer Contributions', 'Total Buffer Subtractions',
                      'Total Credits Issued', 'Total Credits Retired']

In [14]:
cred_group.head()

,Project ID,Issuance Date,Total Buffer Contributions,Total Buffer Subtractions,Total Credits Issued,Total Credits Retired
0,ACR102,2005,0.0,0.0,7984006.0,17666.0
1,ACR103,2005,0.0,0.0,44202.0,28636.0
2,ACR105,2012,1102376.0,1102376.0,2572210.0,2572210.0
3,ACR106,2007,0.0,0.0,138492.0,109452.0
4,ACR107,2008,0.0,0.0,1138.0,127.0


In [15]:
pivot1 = cred_piv.pivot(index='project_id', columns='issuance_date', values='issuance').reset_index()
pivot1 = pivot1.drop(columns=["NaT"])
pivot1.columns = [pivot1.columns[0]] + [f"{col} Issuance" for col in pivot1.columns[1:]]
pivot1.columns.values[0] = "Project ID"
pivot1.fillna(0, inplace=True)

In [16]:
pivot1.head()

,Project ID,2002 Issuance,2004 Issuance,2005 Issuance,2006 Issuance,2007 Issuance,2008 Issuance,2009 Issuance,2010 Issuance,2011 Issuance,2012 Issuance,2013 Issuance,2014 Issuance,2015 Issuance,2016 Issuance,2017 Issuance,2018 Issuance,2019 Issuance,2020 Issuance,2021 Issuance,2022 Issuance,2023 Issuance,2024 Issuance
0,ACR102,0.0,0.0,1898000.0,6086006.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ACR103,0.0,0.0,13848.0,4481.0,0.0,11630.0,0.0,14243.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACR105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2572210.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACR106,0.0,0.0,0.0,0.0,138492.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACR107,0.0,0.0,0.0,0.0,0.0,1138.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
pivot2 = cred_piv.pivot(index='project_id', columns='issuance_date', values='retirement_cancellation').reset_index()
pivot2 = pivot2.drop(columns=["NaT"])
pivot2.columns = [pivot2.columns[0]] + [f"{col} Retirement/Cancellation" for col in pivot2.columns[1:]]
pivot2.columns.values[0] = "Project ID"
pivot2.fillna(0, inplace=True)

In [18]:
pivot2.head()

,Project ID,2002 Retirement/Cancellation,2004 Retirement/Cancellation,2005 Retirement/Cancellation,2006 Retirement/Cancellation,2007 Retirement/Cancellation,2008 Retirement/Cancellation,2009 Retirement/Cancellation,2010 Retirement/Cancellation,2011 Retirement/Cancellation,2012 Retirement/Cancellation,2013 Retirement/Cancellation,2014 Retirement/Cancellation,2015 Retirement/Cancellation,2016 Retirement/Cancellation,2017 Retirement/Cancellation,2018 Retirement/Cancellation,2019 Retirement/Cancellation,2020 Retirement/Cancellation,2021 Retirement/Cancellation,2022 Retirement/Cancellation,2023 Retirement/Cancellation,2024 Retirement/Cancellation
0,ACR102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000.0,1966.0,400.0,10025.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ACR103,0.0,0.0,66.0,0.0,134.0,0.0,0.0,13850.0,10490.0,147.0,3917.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACR105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,645.0,0.0,0.0,2571555.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACR106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54000.0,41330.0,0.0,0.0,14122.0,0.0,0.0
4,ACR107,0.0,0.0,0.0,0.0,0.0,127.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
merged_df = proj1.merge(prev, on='Project ID', how ='outer') \
                  .merge(cred_group, on='Project ID', how='outer') \
                  .merge(pivot1, on='Project ID', how='outer') \
                  .merge(pivot2, on='Project ID', how='outer') \
                  .merge(proj2, on='Project ID', how='outer')

In [45]:
merged_df.head()

,Project ID,Project Name,Voluntary Registry,ARB/WA Project,Voluntary Status,Scope_new,Type_new,Type Rule,Reduction/Removal_new,Methodology/Protocol,Methodology Version,Region,Country,State,Project Site Location,Project Developer,Project Owner,Scope,Type,Reduction/Removal,Issuance Date,Total Buffer Contributions,Total Buffer Subtractions,Total Credits Issued,Total Credits Retired,2002 Issuance,2004 Issuance,2005 Issuance,2006 Issuance,2007 Issuance,2008 Issuance,2009 Issuance,2010 Issuance,2011 Issuance,2012 Issuance,2013 Issuance,2014 Issuance,2015 Issuance,2016 Issuance,2017 Issuance,2018 Issuance,2019 Issuance,2020 Issuance,2021 Issuance,2022 Issuance,2023 Issuance,2024 Issuance,2002 Retirement/Cancellation,2004 Retirement/Cancellation,2005 Retirement/Cancellation,2006 Retirement/Cancellation,2007 Retirement/Cancellation,2008 Retirement/Cancellation,2009 Retirement/Cancellation,2010 Retirement/Cancellation,2011 Retirement/Cancellation,2012 Retirement/Cancellation,2013 Retirement/Cancellation,2014 Retirement/Cancellation,2015 Retirement/Cancellation,2016 Retirement/Cancellation,2017 Retirement/Cancellation,2018 Retirement/Cancellation,2019 Retirement/Cancellation,2020 Retirement/Cancellation,2021 Retirement/Cancellation,2022 Retirement/Cancellation,2023 Retirement/Cancellation,2024 Retirement/Cancellation,Offset Project Operator,Authorized Project Designee,Verifier,Estimated Annual Emission Reductions,ARB ID,ARB Project Type,ARB Project Status,WA ID,WA Project Type,POA ID,Project Listed,Project Registered,Sustainability Certification,Project Type From the Registry,Registry Documents,Project Website,Notes From Registry,Notes from BCTP,Project Description,Added to DB Version
0,ACR1002,Doyon Ts'oo,ACR,No,Listed,NaN,NaN,Improved Forest Management,NaN,ACR Improved Forest Management (IFM) on Non-Fe...,NaN,North America,United States,ALASKA,Southeast and Northwest Interior Alaska,"Doyon, Limited",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not ARB or Ecology Eligible,NaN,NaN,NaN,NaN,NaN,03: Good Health and Well-Being;06: Clean Water...,Forest Carbon,https://acr2.apx.com/mymodule/reg/prjView.asp?...,www.doyon.com/about/sustainability/carbon-fore...,NaN,NaN,NaN,V11 3-2024
1,ACR1006,Challenge Tree Farm IFM Project,ACR,No,Listed,NaN,NaN,Improved Forest Management,NaN,ACR Improved Forest Management (IFM) on Non-Fe...,NaN,North America,United States,CALIFORNIA,Yuba and Butte County,1169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not ARB or Ecology Eligible,NaN,NaN,NaN,NaN,NaN,03: Good Health and Well-Being;06: Clean Water...,Forest Carbon,https://acr2.apx.com/mymodule/reg/prjView.asp?...,NaN,NaN,NaN,NaN,V11 3-2024
2,ACR1007,A-Gas 3-2023B,ACR,No,NaN,NaN,NaN,Ozone Depleting Substances Recovery & Destruction,NaN,ARB Compliance Offset Protocol: Destruction of...,NaN,North America,United States,OHIO,Bowling Green,A-Gas US Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dillon Consulting Limited,NaN,CAOD1007,NaN,Listed - Proposed Project,NaN,NaN,NaN,NaN,NaN,"09: Industry, Innovation and Infrastructure;12...",Ozone Depleting Substances,https://acr2.apx.com/mymodule/reg/prjView.asp?...,www.agasamericas.com,NaN,NaN,NaN,V11 3-2024
3,ACR1008,Advanced Refrigeration - ARS2022004,ACR,No,Listed,NaN,NaN,Advanced Refrigerants,NaN,ACR Advanced Refrigeration Systems,NaN,North America,United States,MISSOURI,"1181 Gannon plaza, Festus, MO",Therm Solutions Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [46]:
merged_df.isnull().sum()

Project ID                                 0
Project Name                              13
Voluntary Registry                        12
ARB/WA Project                            12
Voluntary Status                         335
Scope_new                               9100
Type_new                                9100
Type Rule                               4060
Reduction/Removal_new                   9100
Methodology/Protocol                     113
Methodology Version                     7878
Region                                    22
Country                                   22
State                                   7459
Project Site Location                   7458
Project Developer                         13
Project Owner                           8196
Scope                                    323
Type                                     323
Reduction/Removal                        323
Issuance Date                           4310
Total Buffer Contributions              4310
Total Buff

In [47]:
merged_df.to_csv("Project_Tab.csv")